<a href="https://colab.research.google.com/github/BoyPlankton/public_notebooks/blob/master/2019_FiveThirtyEight_NFL_Forecast_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q trueskill
!pip install --upgrade -q gspread
!pip install --upgrade -q gspread-dataframe

In [0]:
import pandas as pd
import io
import requests

s=requests.get("https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv").content
data=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [0]:
import numpy as np

team_abbr = np.append(data.team1.unique(), data.team2.unique())

team_abbr = np.unique([x for x in team_abbr if isinstance(x, str)])

In [0]:
from trueskill import Rating, rate_1vs1, BETA, global_env
import math
import itertools

teams = {}

for x in team_abbr:
  teams[x] = Rating()

# from the example at https://trueskill.org
def win_probability(team1, team2):
  delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
  sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
  size = len(team1) + len(team2)
  denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
  ts = global_env()
  return ts.cdf(delta_mu / denom)

def eval(x):
  wp_h = win_probability([teams[x["team1"]]], [teams[x["team2"]]])
  wp_v = win_probability([teams[x["team2"]]], [teams[x["team1"]]])

  if wp_h > wp_v:
    return [x["team1"], wp_h]
  elif wp_v > wp_h:
    return [x["team2"], wp_v]

  return ["tie", wp_h]

def upd_ratings(x):
  if x["score1"] > x["score2"]:
    n1, n2 = rate_1vs1(teams[x["team1"]], teams[x["team2"]])
    win = x["team1"]
  elif x["score2"] > x["score1"]:
    n2, n1 = rate_1vs1(teams[x["team2"]], teams[x["team1"]])
    win = x["team2"]
  else:
    n1, n2 = rate_1vs1(teams[x["team1"]], teams[x["team2"]], drawn=True)

  teams[x["team1"]] = n1
  teams[x["team2"]] = n2

def upd_eval(x):
  y = eval(x)

  upd_ratings(x)

  return y

data['wp'] = data[data['date'] <= '2019-11-07'].apply(lambda x: upd_eval(x), axis=1)

In [0]:
data['wp'] = data[data['date'] == '2019-11-11'].apply(lambda x: eval(x), axis=1)

In [38]:
data[data['date'] == '2019-11-11'][['team1','team2','elo_prob1','elo_prob2','score1','score2','wp']]

,team1,team2,elo_prob1,elo_prob2,score1,score2,wp
147,SF,SEA,0.639448,0.360552,NaN,NaN,"[SF, 0.8136098116189436]"
